<a href="https://colab.research.google.com/github/Phunbie/assignments/blob/main/mlzoomcamp8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
link = 'https://github.com/alexeygrigorev/large-datasets/releases/download/dogs-cats/train.zip'

In [2]:
import requests, zipfile, io

In [3]:
r = requests.get(link)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [4]:
import os
import shutil
import glob
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
os.chdir('/content/train')

In [6]:
if os.path.isdir('train/dog') is False:
    os.makedirs('train/dog')
    os.makedirs('train/cat')
    os.makedirs('valid/dog')
    os.makedirs('valid/cat')

    for i in glob.glob('cat*')[:10000]:
        shutil.move(i, 'train/cat')      
    for i in glob.glob('dog*')[:10000]:
        shutil.move(i, 'train/dog')
    for i in glob.glob('cat*')[:2500]:
        shutil.move(i, 'valid/cat')        
    for i in glob.glob('dog*')[:2500]:
        shutil.move(i, 'valid/dog')

os.chdir('../../')

In [7]:
train_path = '/content/train/train'
valid_path = '/content/train/valid'

In [8]:
ImageDataGenerator(rescale=1./255)
train_batches =  ImageDataGenerator(rescale=1./255).flow_from_directory(directory=train_path, target_size=(150,150), batch_size=20, class_mode='binary')

Found 20000 images belonging to 2 classes.


In [9]:
valid_batches =  ImageDataGenerator(rescale=1./255).flow_from_directory(directory=valid_path, target_size=(150,150), batch_size=20, class_mode='binary',shuffle=False)

Found 5000 images belonging to 2 classes.


In [10]:
inputs = keras.Input(shape=(150, 150, 3))
y = Conv2D(filters=32,kernel_size=(3,3),activation='relu')(inputs)
y = MaxPooling2D(2,2)(y)
y = Flatten()(y)
y= Dense(64,activation='relu')(y)
output= Dense(1, activation='sigmoid')(y)
model = Model(inputs=inputs, outputs=output)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                             

In [12]:


optimizer = keras.optimizers.SGD(lr=0.002, momentum=0.8)
loss = keras.losses.BinaryCrossentropy()

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])



/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [13]:
model.fit(
    train_batches,
    steps_per_epoch=100,
    epochs=10,
    validation_data=valid_batches,
    validation_steps=50
)

Epoch 1/10
100/100 [==============================] - 43s 419ms/step - loss: 0.7038 - accuracy: 0.4955 - val_loss: 0.6823 - val_accuracy: 0.6600
Epoch 2/10
100/100 [==============================] - 41s 415ms/step - loss: 0.6895 - accuracy: 0.5415 - val_loss: 0.6398 - val_accuracy: 0.7740
Epoch 3/10
100/100 [==============================] - 42s 418ms/step - loss: 0.6866 - accuracy: 0.5520 - val_loss: 0.7636 - val_accuracy: 0.1930
Epoch 4/10
100/100 [==============================] - 41s 415ms/step - loss: 0.6848 - accuracy: 0.5605 - val_loss: 0.6920 - val_accuracy: 0.4260
Epoch 5/10
100/100 [==============================] - 41s 413ms/step - loss: 0.6788 - accuracy: 0.5675 - val_loss: 0.7916 - val_accuracy: 0.2470
Epoch 6/10
100/100 [==============================] - 41s 411ms/step - loss: 0.6691 - accuracy: 0.5835 - val_loss: 0.7157 - val_accuracy: 0.4260
Epoch 7/10
100/100 [==============================] - 41s 411ms/step - loss: 0.6602 - accuracy: 0.6050 - val_loss: 0.4021 - val_ac

In [15]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [16]:
train_ds = train_gen.flow_from_directory(directory=train_path, target_size=(150,150), batch_size=20, class_mode='binary')

Found 20000 images belonging to 2 classes.


In [17]:
model.fit(
    train_ds,
    steps_per_epoch=100,
    epochs=10,
    validation_data=valid_batches,
    validation_steps=50
)

Epoch 1/10
100/100 [==============================] - 50s 503ms/step - loss: 0.6729 - accuracy: 0.5810 - val_loss: 0.5966 - val_accuracy: 0.6930
Epoch 2/10
100/100 [==============================] - 51s 507ms/step - loss: 0.6613 - accuracy: 0.5990 - val_loss: 0.8311 - val_accuracy: 0.3260
Epoch 3/10
100/100 [==============================] - 50s 502ms/step - loss: 0.6619 - accuracy: 0.5945 - val_loss: 0.6997 - val_accuracy: 0.4970
Epoch 4/10
100/100 [==============================] - 50s 498ms/step - loss: 0.6719 - accuracy: 0.5800 - val_loss: 0.8035 - val_accuracy: 0.2940
Epoch 5/10
100/100 [==============================] - 50s 499ms/step - loss: 0.6618 - accuracy: 0.5990 - val_loss: 0.7805 - val_accuracy: 0.3630
Epoch 6/10
100/100 [==============================] - 50s 496ms/step - loss: 0.6689 - accuracy: 0.5855 - val_loss: 0.7624 - val_accuracy: 0.4110
Epoch 7/10
100/100 [==============================] - 50s 497ms/step - loss: 0.6607 - accuracy: 0.6035 - val_loss: 0.7098 - val_ac